# Implementing market basket analysis

In [1]:
#Loading neccesary packages
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [40]:
#Reading Data From Web
bookclub = pd.read_csv("/Users/superman/Desktop/Education/MBA/Semester-V/BUSI 650/course_files_export/CharlesBookClub.csv")
bookclub

,Seq#,ID#,Gender,M,R,F,FirstPurch,ChildBks,YouthBks,CookBks,...,ItalCook,ItalAtlas,ItalArt,Florence,Related Purchase,Mcode,Rcode,Fcode,Yes_Florence,No_Florence
0,1,25,1,297,14,2,22,0,1,1,...,0,0,0,0,0,5,4,2,0,1
1,2,29,0,128,8,2,10,0,0,0,...,0,0,0,0,0,4,3,2,0,1
2,3,46,1,138,22,7,56,2,1,2,...,1,0,0,0,2,4,4,3,0,1
3,4,47,1,228,2,1,2,0,0,0,...,0,0,0,0,0,5,1,1,0,1
4,5,51,1,257,10,1,10,0,0,0,...,0,0,0,0,0,5,3,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,3996,32950,0,141,2,2,6,0,0,0,...,0,0,0,0,0,4,1,2,0,1
3996,3997,32954,1,357,16,9,52,1,1,2,...,0,0,0,0,1,5,4,3,0,1
3997,3998,32955,0,48,12,1,12,0,0,0,...,0,0,0,0,0,2,3,1,0,1
3998,3999,32976,0,214,14,7,38,1,1,3,...,0,0,0,0,0,5,4,3,0,1


In [67]:
bookclub.tail()

,Seq#,ID#,Gender,M,R,F,FirstPurch,ChildBks,YouthBks,CookBks,...,ItalCook,ItalAtlas,ItalArt,Florence,Related Purchase,Mcode,Rcode,Fcode,Yes_Florence,No_Florence
3995,3996,32950,0,141,2,2,6,0,0,0,...,0,0,0,0,0,4,1,2,0,1
3996,3997,32954,1,357,16,9,52,1,1,2,...,0,0,0,0,1,5,4,3,0,1
3997,3998,32955,0,48,12,1,12,0,0,0,...,0,0,0,0,0,2,3,1,0,1
3998,3999,32976,0,214,14,7,38,1,1,3,...,0,0,0,0,0,5,4,3,0,1
3999,4000,32977,1,114,10,2,12,0,0,0,...,0,0,0,0,1,4,3,2,0,1


In [56]:
bookclubnew_df = bookclub.drop((['Seq#','M','R','F','FirstPurch','Gender','Related Purchase',
       'Mcode', 'Rcode', 'Fcode', 'Yes_Florence', 'No_Florence']), axis =1)
bookclubnew_df

,ID#,ChildBks,YouthBks,CookBks,DoItYBks,RefBks,ArtBks,GeogBks,ItalCook,ItalAtlas,ItalArt,Florence
0,25,0,1,1,0,0,0,0,0,0,0,0
1,29,0,0,0,0,0,0,0,0,0,0,0
2,46,2,1,2,0,1,0,1,1,0,0,0
3,47,0,0,0,0,0,0,0,0,0,0,0
4,51,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3995,32950,0,0,0,0,0,0,0,0,0,0,0
3996,32954,1,1,2,2,2,0,1,0,0,0,0
3997,32955,0,0,0,0,0,0,0,0,0,0,0
3998,32976,1,1,3,1,0,0,0,0,0,0,0


In [57]:
bookclubnew_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
ID#,4000.0,16594.62300,9484.433792,25.0,8253.25,16581.0,24838.25,32977.0
ChildBks,4000.0,0.63975,0.994343,0.0,0.00,0.0,1.00,7.0
YouthBks,4000.0,0.30475,0.611940,0.0,0.00,0.0,0.00,5.0
CookBks,4000.0,0.73125,1.089413,0.0,0.00,0.0,1.00,7.0
DoItYBks,4000.0,0.35075,0.687999,0.0,0.00,0.0,1.00,5.0
RefBks,4000.0,0.25625,0.558269,0.0,0.00,0.0,0.00,4.0
ArtBks,4000.0,0.28900,0.600890,0.0,0.00,0.0,0.00,5.0
GeogBks,4000.0,0.38750,0.750656,0.0,0.00,0.0,1.00,6.0
ItalCook,4000.0,0.12525,0.385486,0.0,0.00,0.0,0.00,3.0
ItalAtlas,4000.0,0.03750,0.214721,0.0,0.00,0.0,0.00,2.0


In [58]:
bookclubnew_df.dtypes

ID#          int64
ChildBks     int64
YouthBks     int64
CookBks      int64
DoItYBks     int64
RefBks       int64
ArtBks       int64
GeogBks      int64
ItalCook     int64
ItalAtlas    int64
ItalArt      int64
Florence     int64
dtype: object

In [59]:
bookclubnew_df.shape

(4000, 12)

In [60]:
bookclubnew_df.columns

Index(['ID#', 'ChildBks', 'YouthBks', 'CookBks', 'DoItYBks', 'RefBks',
       'ArtBks', 'GeogBks', 'ItalCook', 'ItalAtlas', 'ItalArt', 'Florence'],
      dtype='object')

In [61]:
bookclubnew_df.columns.str.strip() #removes spaces from beginning and end
bookclubnew_df.columns

Index(['ID#', 'ChildBks', 'YouthBks', 'CookBks', 'DoItYBks', 'RefBks',
       'ArtBks', 'GeogBks', 'ItalCook', 'ItalAtlas', 'ItalArt', 'Florence'],
      dtype='object')

In [62]:
bookclubnew_df.head()

,ID#,ChildBks,YouthBks,CookBks,DoItYBks,RefBks,ArtBks,GeogBks,ItalCook,ItalAtlas,ItalArt,Florence
0,25,0,1,1,0,0,0,0,0,0,0,0
1,29,0,0,0,0,0,0,0,0,0,0,0
2,46,2,1,2,0,1,0,1,1,0,0,0
3,47,0,0,0,0,0,0,0,0,0,0,0
4,51,0,0,0,0,0,0,0,0,0,0,0


In [63]:
bookclub_df = pd.get_dummies(bookclubnew_df, prefix_sep='_', drop_first=False)
bookclub_df.head()

,ID#,ChildBks,YouthBks,CookBks,DoItYBks,RefBks,ArtBks,GeogBks,ItalCook,ItalAtlas,ItalArt,Florence
0,25,0,1,1,0,0,0,0,0,0,0,0
1,29,0,0,0,0,0,0,0,0,0,0,0
2,46,2,1,2,0,1,0,1,1,0,0,0
3,47,0,0,0,0,0,0,0,0,0,0,0
4,51,0,0,0,0,0,0,0,0,0,0,0


In [64]:
new_bookclub = bookclub_df.set_index('ID#')
new_bookclub

,ChildBks,YouthBks,CookBks,DoItYBks,RefBks,ArtBks,GeogBks,ItalCook,ItalAtlas,ItalArt,Florence
ID#,,,,,,,,,,,
25,0,1,1,0,0,0,0,0,0,0,0
29,0,0,0,0,0,0,0,0,0,0,0
46,2,1,2,0,1,0,1,1,0,0,0
47,0,0,0,0,0,0,0,0,0,0,0
51,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
32950,0,0,0,0,0,0,0,0,0,0,0
32954,1,1,2,2,2,0,1,0,0,0,0
32955,0,0,0,0,0,0,0,0,0,0,0


In [72]:
#converting all positive vaues to 1 and everything else to 0
def my_encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

new1_bookclub= new_bookclub.applymap(my_encode_units)

# Data Preparation

# Training Model

In [74]:
#Generatig frequent itemsets
my_frequent_itemsets = apriori(new1_bookclub, min_support=0.07, use_colnames=True)

# http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/

In [75]:
#generating rules
my_rules = association_rules(my_frequent_itemsets)

In [76]:
#viewing top 100 rules
my_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(YouthBks, ChildBks)",(CookBks),0.14750,0.4155,0.12000,0.813559,1.958025,0.058714,3.135045
1,"(RefBks, ChildBks)",(CookBks),0.12825,0.4155,0.10350,0.807018,1.942280,0.050212,3.028773
2,"(YouthBks, DoItYBks)",(CookBks),0.10325,0.4155,0.08375,0.811138,1.952197,0.040850,3.094853
3,"(RefBks, DoItYBks)",(CookBks),0.09250,0.4155,0.07450,0.805405,1.938400,0.036066,3.003681


# Making recommendations

In [78]:
new1_bookclub.head()

,ChildBks,YouthBks,CookBks,DoItYBks,RefBks,ArtBks,GeogBks,ItalCook,ItalAtlas,ItalArt,Florence
ID#,,,,,,,,,,,
25,0,1,1,0,0,0,0,0,0,0,0
29,0,0,0,0,0,0,0,0,0,0,0
46,1,1,1,0,1,0,1,1,0,0,0
47,0,0,0,0,0,0,0,0,0,0,0
51,0,0,0,0,0,0,0,0,0,0,0


In [80]:
#Filtering rules based on condition
my_rules[ (my_rules['lift'] >= 3) &
       (my_rules['confidence'] >= 0.3) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
